In [1]:
import sys

In [2]:
from platform import python_version
print (f"current platform python version: {python_version()}")

current platform python version: 3.11.6


In [3]:
#!{sys.executable} -m pip install --upgrade --user kfp==2.4.0 kfp-server-api==2.0.3 kfp-pipeline-spec==0.2.2
#!{sys.executable} -m pip install --upgrade --user kfp-kubernetes==1.0.0

In [4]:
!{sys.executable} -m pip list | grep kfp

kfp                           2.4.0
kfp-kubernetes                1.0.0
kfp-pipeline-spec             0.2.2
kfp-server-api                2.0.3


In [5]:
from kfp.client import Client
from kfp import dsl
import kfp


@dsl.component(
    base_image='python:3.10.13-bullseye'    
)
def add(a: float, b: float) -> float:
    '''Calculates sum of two arguments'''
    return a + b


@dsl.pipeline(
    name='Addition pipeline',
    description='An example pipeline that performs addition calculations.')
def add_pipeline(
    a: float = 1.0,
    b: float = 7.0,
):
    first_add_task = add(a=a, b=4.0)
    second_add_task = add(a=first_add_task.output, b=b)

    
client = Client()
NAMESPACE = client.get_user_namespace()
EXPERIMENT_NAME = "demo"
ENABLE_CACHING = False
print(NAMESPACE)

kubeflow-kindfor


/opt/conda/lib/python3.11/site-packages/kfp/client/client.py:159: FutureWarning: This client only works with Kubeflow Pipeline v2.0.0-beta.2 and later versions.
  warnings.warn(


In [6]:
client.create_run_from_pipeline_func(
    pipeline_func = add_pipeline, 
    experiment_name = EXPERIMENT_NAME,
    namespace = NAMESPACE,
    enable_caching=ENABLE_CACHING,
    # mode=kfp.dsl.PipelineExecutionMode.V2_COMPATIBLE,
    arguments={
        'a': 7.0,
        'b': 8.0
    })

RunPipelineResult(run_id=ac863cbb-18cc-4fdb-9a35-bdc8852b9369)